# 준비

먼저 Naver Cloud Platform에서 지오코딩 API를 사용할 수 있도록 API Key를 받습니다.  
https://apidocs.ncloud.com/ko/ai-naver/maps_geocoding/  

In [47]:
# 데이터 처리
import numpy as np
import pandas as pd

In [48]:
# "전국CCTV표준데이터" CSV 파일 로딩 
df_cctv = pd.read_csv('./data/전국CCTV표준데이터.csv', encoding="EUC-KR", header=0, 
                           names=['admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 'pixel', 'direction', 'period', 'date_install', 'tel', 'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'], 
                           dtype={'cctv_cnt':object, 'pixel':object, 'period':object, 'lat':float, 'lon':float, 'offer_admin_cd':object}, 
                  thousands = ',', #천단위 쉼표 제거   
                  index_col=None) 

In [49]:
df_cctv.dropna(subset=['lat'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['lon'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv.dropna(subset=['addr_rd'],axis=0,inplace=True) # 도로명주소 nulll 데이터 삭제
df_cctv.dropna(subset=['addr_jb'],axis=0,inplace=True) #지번주소 null 데이터 삭제 

In [65]:
# 5만 건 중 0.1% 샘플링
df_cctv2 = df_cctv.sample(frac=0.001)
df_cctv2.info()
df_cctv2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 31746 to 14741
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   admin_nm        24 non-null     object 
 1   addr_rd         24 non-null     object 
 2   addr_jb         24 non-null     object 
 3   purpose         24 non-null     object 
 4   cctv_cnt        24 non-null     object 
 5   pixel           24 non-null     object 
 6   direction       17 non-null     object 
 7   period          24 non-null     object 
 8   date_install    21 non-null     object 
 9   tel             24 non-null     object 
 10  lat             24 non-null     float64
 11  lon             24 non-null     float64
 12  date_make       24 non-null     object 
 13  offer_admin_cd  24 non-null     object 
 14  offer_admin_nm  24 non-null     object 
dtypes: float64(2), object(13)
memory usage: 3.0+ KB


,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구
7419,서울특별시 마포구청,와우산로30길,창전동 3-183,방범(공원),1,200,360도전방면,30,2012-07,02-3153-8432,37.552997,126.930324,2020-05-20,3130000,서울특별시 마포구


# Geopy - Nominatim을 이용한 지오코딩  
### Geopy  
- 지오코딩 패키지 (https://geopy.readthedocs.io/en/stable/#  )       
- Geopy는 Nominatim 뿐만 아니라 Google 등 다양한 지오코딩 서비스를 지원함 (다만 한국 포털 지도 API는 아직 없는 것으로 보임)   

### Nominatim   
- OpenStreetMap의 지명이나 주소를 이용하여 지오코딩을 할 수 있는 서비스   
 : https://nominatim.org/  
 : https://wiki.openstreetmap.org/wiki/Ko:Nominatim  
 : https://github.com/osm-search/Nominatim 

In [51]:
import geopy  #!pip install geopy
from geopy.geocoders import Nominatim
from  geopy.extra.rate_limiter import RateLimiter 
geolocator = Nominatim(user_agent='South Korea') 
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=2) 

In [66]:
df_cctv3 = df_cctv2.copy()

In [53]:
df_cctv3['temp'] = df_cctv3['addr_rd'].apply(geocode_with_delay) 
df_cctv3.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,temp
34858,홍천군청,강원도 홍천군 홍천읍 갈마곡리 507-7,강원도 홍천군 홍천읍 갈마곡리 507-7(남산교 앞),생활방범,1,200,NaN,30,2017-06,033-430-2301,37.687995,127.890047,2019-06-30,4250000,강원도 홍천군,None
15478,대구광역시 중구청,대구광역시 중구 대봉1동 44-53,대구광역시 중구 대봉로 228-16,생활방범,1,200,360도 전방면,30,2018-04,053-661-2494,35.858984,128.602272,2018-06-30,3410000,대구광역시 중구,None
11041,서울특별시 강북구청,서울특별시 강북구 한천로148길 28-9,서울특별시 강북구 수유동 180-19,생활방범,1,200,NaN,30,2018-01,02-901-7266,37.642994,127.025307,2020-05-12,3080000,서울특별시 강북구,"(한천로148길, 수유3동, 강북구, 서울, 01065, 대한민국, (37.6427..."
36716,부산광역시 북구청,부산광역시 북구 만덕대로102번가길 25,부산광역시 북구 덕천동 389-17,생활방범,1,200,360도 전방면,30,2015-12,051-309-2031,35.212544,129.017432,2020-03-20,3320000,부산광역시 북구,"(만덕대로102번가길, 덕천1동, 덕천동, 북구, 부산, 46568, 대한민국, (..."
5171,서울특별시 구로구청,서울특별시 구로구 구로동로25길 5-10,서울특별시 구로구 구로동 728-19,생활방범,3,200,NaN,30,2007-09,02-860-3437,37.490125,126.883490,2020-10-23,3160000,서울특별시 구로구,None


In [54]:
df_cctv3["coords"] = df_cctv3['temp'].apply(lambda loc: tuple(loc.point) if loc else None) 
df_cctv3

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,temp,coords
34858,홍천군청,강원도 홍천군 홍천읍 갈마곡리 507-7,강원도 홍천군 홍천읍 갈마곡리 507-7(남산교 앞),생활방범,1,200,NaN,30,2017-06,033-430-2301,37.687995,127.890047,2019-06-30,4250000,강원도 홍천군,None,None
15478,대구광역시 중구청,대구광역시 중구 대봉1동 44-53,대구광역시 중구 대봉로 228-16,생활방범,1,200,360도 전방면,30,2018-04,053-661-2494,35.858984,128.602272,2018-06-30,3410000,대구광역시 중구,None,None
11041,서울특별시 강북구청,서울특별시 강북구 한천로148길 28-9,서울특별시 강북구 수유동 180-19,생활방범,1,200,NaN,30,2018-01,02-901-7266,37.642994,127.025307,2020-05-12,3080000,서울특별시 강북구,"(한천로148길, 수유3동, 강북구, 서울, 01065, 대한민국, (37.6427...","(37.6427285, 127.0242983, 0.0)"
36716,부산광역시 북구청,부산광역시 북구 만덕대로102번가길 25,부산광역시 북구 덕천동 389-17,생활방범,1,200,360도 전방면,30,2015-12,051-309-2031,35.212544,129.017432,2020-03-20,3320000,부산광역시 북구,"(만덕대로102번가길, 덕천1동, 덕천동, 북구, 부산, 46568, 대한민국, (...","(35.2124919, 129.0175789, 0.0)"
5171,서울특별시 구로구청,서울특별시 구로구 구로동로25길 5-10,서울특별시 구로구 구로동 728-19,생활방범,3,200,NaN,30,2007-09,02-860-3437,37.490125,126.883490,2020-10-23,3160000,서울특별시 구로구,None,None
20650,경기도 이천시청,경기도 이천시 마장면 덕평로923번길 11-5,경기도 이천시 마장면 덕평리 332-7,생활방범,1,200,360,30,2013-05,031-644-2942,37.239982,127.362302,2020-01-20,4070000,경기도 이천시,None,None
6491,인천광역시 연수구청,인천광역시 연수구 한진로 30,인천광역시 연수구 옥련동 232,생활방범,2,300,360도 전방면,30,NaN,032-749-8885,37.413891,126.677723,2020-07-07,3520000,인천광역시 연수구,None,None
45164,서울특별시 강동구청,서울특별시 강동구 양재대로 1557 (천호동),서울특별시 강동구 천호동 44,생활방범,4,200,360도전방면,30,2017-11,02-3425-7090,37.545074,127.142494,2020-09-11,3240000,서울특별시 강동구,None,None
13356,부산광역시 중구청,부산광역시 중구 대청북길 10,부산광역시 중구 대청동4가 75-176,다목적,2,200,360도 전방면,30,2019-12,051-600-4172,35.105219,129.028422,2020-06-25,3250000,부산광역시 중구,None,None
17792,부산광역시 금정구청,부산광역시 금정구 중앙대로 2002,부산광역시 금정구 남산동 225-13,생활방범,1,200,360도 전방면,30,2020-11,051-519-5731,35.262885,129.092239,2020-08-12,3350000,부산광역시 금정구,None,None


In [55]:
# 좌표(coords)가 없는 데이터 삭제
df_cctv3.dropna(subset=['coords'],axis=0,inplace=True)

In [56]:
df_cctv3[['y','x','z']] = pd.DataFrame(df_cctv3['coords'].tolist(), index=df_cctv2.index)
df_cctv3

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,temp,coords,y,x,z
11041,서울특별시 강북구청,서울특별시 강북구 한천로148길 28-9,서울특별시 강북구 수유동 180-19,생활방범,1,200,NaN,30,2018-01,02-901-7266,37.642994,127.025307,2020-05-12,3080000,서울특별시 강북구,"(한천로148길, 수유3동, 강북구, 서울, 01065, 대한민국, (37.6427...","(37.6427285, 127.0242983, 0.0)",37.642728,127.024298,0.0
36716,부산광역시 북구청,부산광역시 북구 만덕대로102번가길 25,부산광역시 북구 덕천동 389-17,생활방범,1,200,360도 전방면,30,2015-12,051-309-2031,35.212544,129.017432,2020-03-20,3320000,부산광역시 북구,"(만덕대로102번가길, 덕천1동, 덕천동, 북구, 부산, 46568, 대한민국, (...","(35.2124919, 129.0175789, 0.0)",35.212492,129.017579,0.0
11825,서울특별시 강북구청,서울특별시 강북구 삼양로113길 5,서울특별시 강북구 인수동 342-4,생활방범,1,200,NaN,30,2010-03,02-901-7266,37.642719,127.016136,2020-05-12,3080000,서울특별시 강북구,"(삼양로113길, 인수동, 강북구, 서울, 01021, 대한민국, (37.64257...","(37.6425723, 127.0162158, 0.0)",37.642572,127.016216,0.0


### 원 좌표와 네이버에서 구한 좌표와의 거리 차를 검토

In [26]:
# 거리를 계산하기 위한 라이브러리 로드
from geopy.distance import great_circle

In [57]:
# Haversine식을 이용한 거리 계산
##great_circle( (출발지 위도, 출발지 경도), (도착지 위도, 도착지 경도) )
df_cctv3['dist'] = df_cctv3.apply(lambda x: great_circle((x['lat'], x['lon']), (x['y'], x['x'])).meters, axis=1)
df_cctv3.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,...,lon,date_make,offer_admin_cd,offer_admin_nm,temp,coords,y,x,z,dist
11041,서울특별시 강북구청,서울특별시 강북구 한천로148길 28-9,서울특별시 강북구 수유동 180-19,생활방범,1,200,NaN,30,2018-01,02-901-7266,...,127.025307,2020-05-12,3080000,서울특별시 강북구,"(한천로148길, 수유3동, 강북구, 서울, 01065, 대한민국, (37.6427...","(37.6427285, 127.0242983, 0.0)",37.642728,127.024298,0.0,93.564594
36716,부산광역시 북구청,부산광역시 북구 만덕대로102번가길 25,부산광역시 북구 덕천동 389-17,생활방범,1,200,360도 전방면,30,2015-12,051-309-2031,...,129.017432,2020-03-20,3320000,부산광역시 북구,"(만덕대로102번가길, 덕천1동, 덕천동, 북구, 부산, 46568, 대한민국, (...","(35.2124919, 129.0175789, 0.0)",35.212492,129.017579,0.0,14.585783
11825,서울특별시 강북구청,서울특별시 강북구 삼양로113길 5,서울특별시 강북구 인수동 342-4,생활방범,1,200,NaN,30,2010-03,02-901-7266,...,127.016136,2020-05-12,3080000,서울특별시 강북구,"(삼양로113길, 인수동, 강북구, 서울, 01021, 대한민국, (37.64257...","(37.6425723, 127.0162158, 0.0)",37.642572,127.016216,0.0,17.781819


# 상용 서비스를 활용

In [38]:
import urllib
import json

In [39]:
# 네이버 API 정보 읽기 
f = open('../../etc/naver.txt', 'r')
lines = f.readlines()

api_id = lines[0].replace("\n", "")
api_key = lines[1].replace("\n", "")

In [40]:
# 네이버 Map API https://console.ncloud.com/mc/solution/naverService/application
def search_map(search_text):
    client_id = 'id'
    client_secret = 'secret'
    encText = urllib.parse.quote(str(search_text))
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query='+encText
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', api_id) 
    request.add_header('X-NCP-APIGW-API-KEY', api_key) 
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        print(response_body.decode('utf-8'))
        return response_body.decode('utf-8')
    else:
        print("Error Code:" + rescode)

In [16]:
# 단 건 테스트
x = []
y = []

temp_map = search_map('경기 부천시  소사본1동 181-26 2층 201호')  #NAVER search_map 함수 실행
temp_map = json.loads(temp_map)
try:
    #print(temp_map)
    temp_map = temp_map['addresses'][0]
    x = temp_map['x']
    y = temp_map['y']
except IndexError:
    pass

print(x, y)

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"","jibunAddress":"경기도 부천시 소사본동 181-26","englishAddress":"181-26, Sosabon-dong, Bucheon-si, Gyeonggi-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"경기도","shortName":"경기도","code":""},{"types":["SIGUGUN"],"longName":"부천시","shortName":"부천시","code":""},{"types":["DONGMYUN"],"longName":"소사본동","shortName":"소사본동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NUMBER"],"longName":"","shortName":"","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"181-26","shortName":"181-26","code":""},{"types":["POSTAL_CODE"],"longName":"","shortName":"","code":""}],"x":"126.7953156","y":"37.4748725","distance":0.0}],"errorMessage":""}
126.7953156 37.4748725


In [67]:
x1 = []
y1 = []
length_df = len(df_cctv2)
print(length_df)

24


In [68]:
# 데이터프레임 행수만큼 읽으면서 addr 항목의 주소를 Naver API로 돌려서 좌표값을 받아 리스트에 추가  
for i in range(length_df):
    addr = df_cctv2.iloc[i]['addr_rd']
    #print(addr)
    #print(i)
    temp_map = search_map(addr)  #NAVER search_map 함수 실행
    temp_map = json.loads(temp_map)
    try:
        #print(temp_map)
        temp_map = temp_map['addresses'][0]
        x = temp_map['x']
        y = temp_map['y']
        x1.append(x)
        y1.append(y)    
    except:
        x1.append(0)
        y1.append(0)    

{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[{"roadAddress":"전라북도 군산시 삼화1길 7","jibunAddress":"전라북도 군산시 문화동 898-11","englishAddress":"7, Samhwa 1-gil, Gunsan-si, Jeollabuk-do, Republic of Korea","addressElements":[{"types":["SIDO"],"longName":"전라북도","shortName":"전라북도","code":""},{"types":["SIGUGUN"],"longName":"군산시","shortName":"군산시","code":""},{"types":["DONGMYUN"],"longName":"문화동","shortName":"문화동","code":""},{"types":["RI"],"longName":"","shortName":"","code":""},{"types":["ROAD_NAME"],"longName":"삼화1길","shortName":"삼화1길","code":""},{"types":["BUILDING_NUMBER"],"longName":"7","shortName":"7","code":""},{"types":["BUILDING_NAME"],"longName":"","shortName":"","code":""},{"types":["LAND_NUMBER"],"longName":"898-11","shortName":"898-11","code":""},{"types":["POSTAL_CODE"],"longName":"54122","shortName":"54122","code":""}],"x":"126.7063812","y":"35.9742544","distance":0.0}],"errorMessage":""}
{"status":"OK","meta":{"totalCount":1,"page":1,"count":1},"addresses":[

In [69]:
# 좌표 리스트 값을 데이터프레임의 X, Y 항목에 넣어줌 
df_cctv2['x'] = x1
df_cctv2['y'] = y1

In [70]:
# 잘못 입력된 좌표 리턴값 "[]"을 치환
df_cctv2['x'] = df_cctv2['x'].str.replace('[', '0')
df_cctv2['x'] = df_cctv2['x'].str.replace(']', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace('[', '0')
df_cctv2['y'] = df_cctv2['y'].str.replace(']', '0')
df_cctv2.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.7063812,35.9742544
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.0885535,37.6239886
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.3741981,36.4079271
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.4285288,35.5045087
7419,서울특별시 마포구청,와우산로30길,창전동 3-183,방범(공원),1,200,360도전방면,30,2012-07,02-3153-8432,37.552997,126.930324,2020-05-20,3130000,서울특별시 마포구,NaN,NaN


In [71]:
# 좌표 항목을 실수 형식으로 변환 
df_cctv2['x'] = df_cctv2['x'].astype(float)
df_cctv2['y'] = df_cctv2['y'].astype(float)

In [72]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.706381,35.974254
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.088554,37.623989
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.374198,36.407927
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.428529,35.504509
7419,서울특별시 마포구청,와우산로30길,창전동 3-183,방범(공원),1,200,360도전방면,30,2012-07,02-3153-8432,37.552997,126.930324,2020-05-20,3130000,서울특별시 마포구,NaN,NaN
25390,부산광역시 수영구청,부산광역시 수영구 장대골로27번길 29,부산광역시 수영구 광안동 735-11,생활방범,1,200,360도전방면,30,2015-11,051-610-4492,35.157912,129.110367,2019-12-20,3380000,부산광역시 수영구,129.110358,35.157911
41489,전라북도 전주시청,전라북도 전주시 완산구 안행2길 25,전라북도 전주시 완산구 삼천동1가 739,어린이보호,2,200,반경100m이내,30,2012-12,063-281-2072,35.802009,127.129149,2020-08-31,4640000,전라북도 전주시,127.128724,35.801831
11684,서울특별시 강북구청,서울특별시 강북구 삼양로139가길 43,서울특별시 강북구 우이동 73-23,생활방범,1,200,NaN,30,2012-05,02-901-7266,37.651230,127.012093,2020-05-12,3080000,서울특별시 강북구,127.012015,37.651108
24365,안전관리과,경상북도 칠곡군 기산면 행정리 1180-1,경상북도 칠곡군 기산면 행정리 1180-1,차량방범,1,200,NaN,30,2014-07,054-979-6881,35.969625,128.363002,2018-08-22,5220000,경상북도 칠곡군,128.363010,35.969626
29014,서울특별시 성동구청,서울특별시 성동구 자동차시장길 22,서울특별시 성동구 용답동 223-5,생활방범,3,200,360도전방면,30,2014-09,02-2286-5879,37.561981,127.057297,2020-12-04,3030000,서울특별시 성동구,127.056381,37.561185


In [73]:
df_cctv2.dropna(subset=['x'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제
df_cctv2.dropna(subset=['y'],axis=0,inplace=True)  #거리 비교를 위해 null 삭제

### 원 좌표와 네이버에서 구한 좌표와의 거리 차를 검토

In [74]:
# Haversine식을 이용한 거리 계산
##great_circle( (출발지 위도, 출발지 경도), (도착지 위도, 도착지 경도) )
df_cctv2['dist'] = df_cctv2.apply(lambda x: great_circle((x['lat'], x['lon']), (x['y'], x['x'])).meters, axis=1)

In [75]:
df_cctv2

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,x,y,dist
31746,전라북도 군산시,전라북도 군산시 삼화1길 7,전라북도 군산시 문화동 921,생활방범,1,200,360도 전방면,30,2015-06,063-454-7922,35.974253,126.706394,2020-03-12,4670000,전라북도 군산시,126.706381,35.974254,1.208316
8454,서울특별시 노원구청,서울특별시 노원구 화랑로51길 17 솔밭근린공원,서울특별시 노원구 공릉동 111-1 (솔밭근린공원 -1),시설물관리,1,200,회전,30,2015-09,02-2116-3952,37.622720,127.088171,2019-04-08,3100000,서울특별시 노원구,127.088554,37.623989,145.004489
43019,경상북도 영덕군청,경상북도 영덕군 영덕읍 우곡리 609,경상북도 영덕군 영덕읍 우곡리 609,생활방범,1,200,단방향,30,2016-06,054-730-6300,36.408073,129.373656,2019-09-17,5180000,경상북도 영덕군,129.374198,36.407927,51.153852
12573,울산광역시 동구청,울산광역시 동구 진성9길 2,울산광역시 동구 일산동 456-9,생활방범,1,200,대송로 10 방향,30,2017-08,052-209-3145,35.504506,129.428526,2020-10-30,3710000,울산광역시 동구,129.428529,35.504509,0.437615
25390,부산광역시 수영구청,부산광역시 수영구 장대골로27번길 29,부산광역시 수영구 광안동 735-11,생활방범,1,200,360도전방면,30,2015-11,051-610-4492,35.157912,129.110367,2019-12-20,3380000,부산광역시 수영구,129.110358,35.157911,0.783797
41489,전라북도 전주시청,전라북도 전주시 완산구 안행2길 25,전라북도 전주시 완산구 삼천동1가 739,어린이보호,2,200,반경100m이내,30,2012-12,063-281-2072,35.802009,127.129149,2020-08-31,4640000,전라북도 전주시,127.128724,35.801831,43.109825
11684,서울특별시 강북구청,서울특별시 강북구 삼양로139가길 43,서울특별시 강북구 우이동 73-23,생활방범,1,200,NaN,30,2012-05,02-901-7266,37.651230,127.012093,2020-05-12,3080000,서울특별시 강북구,127.012015,37.651108,15.208931
24365,안전관리과,경상북도 칠곡군 기산면 행정리 1180-1,경상북도 칠곡군 기산면 행정리 1180-1,차량방범,1,200,NaN,30,2014-07,054-979-6881,35.969625,128.363002,2018-08-22,5220000,경상북도 칠곡군,128.363010,35.969626,0.696153
29014,서울특별시 성동구청,서울특별시 성동구 자동차시장길 22,서울특별시 성동구 용답동 223-5,생활방범,3,200,360도전방면,30,2014-09,02-2286-5879,37.561981,127.057297,2020-12-04,3030000,서울특별시 성동구,127.056381,37.561185,119.795268
43993,서울특별시 금천구,서울특별시 금천구 독산로44길 51,서울특별시 금천구 시흥4동 793-6,다목적,4,200,360도방향,30,NaN,02-2627-1903,37.465061,126.907546,2020-07-31,3170000,서울특별시 금천구,126.907544,37.461831,359.160173


In [76]:
df_cctv2['dist'].min()

0.43761513347072556

In [77]:
df_cctv2['dist'].max()

359.160172916656

In [79]:
df_cctv2['dist'].mean()

46.103398054641914

In [78]:
df_cctv2['dist'].median()

15.208931047570399

포털지도와 구글맵 등을 이용하여 도로명주소, 지번주소, 원 좌표, 도로명주소 좌표 등을 비교

In [81]:
#다음 과정을 위해 데이터프레임을 csv 파일로 저장 
df_cctv2.to_csv("./data/cctv_coord_sample.csv", encoding='euc-kr')